<a href="https://colab.research.google.com/github/andreunilux/BSP-6/blob/master/BSP6_V2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install missing dependancies
!pip install transformers accelerate

#Library used for fine tuning

from torch.utils.data import DataLoader, Dataset
from transformers import Trainer, TrainingArguments

# Pandas Dataframe Library
import json
import pandas as pd
import os
from IPython.display import display
import numpy as np 

# HateBert Libarary
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, BertConfig, BertTokenizer


from google.colab import drive
drive.mount('/content/drive')
os.getcwd()
os.chdir('/content/drive/MyDrive/BSP6')   

current_model="bert-base-uncased"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:

train_data = pd.read_json('train.json', lines=True)
val_data = pd.read_json('val.json', lines=True)
test_data = pd.read_json('test.json', lines=True)






In [3]:
from IPython.core.interactiveshell import dis
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained(current_model, do_lower_case=True)

def display_dataset(dataset):
  print("Look at example representative entries of the dataset")
  display(dataset.head(2))
  print("")
  print("Is the dataset complete or are some values missing?")
  display(dataset.count())
  print("")
  print("How balanced is the dataset?")
  display(dataset.label.value_counts(normalize=True))
  print("")

  tokens_c=[]
  length_of_tokens_c=[]
  for sent in dataset.context:
    input_ids = tokenizer.encode(sent,add_special_tokens=True)
    tokens_c.append(input_ids)
    length_of_tokens_c.append(len(input_ids))
  dataset['tokens_c']= tokens_c   
  dataset['length_of_tokens_c']= length_of_tokens_c 
  print("")
  print("token length of the context sentences")
  display(dataset.length_of_tokens_c.describe())
 
  tokens_t=[]
  length_of_tokens_t=[]
  for sent in dataset.target:
    input_ids = tokenizer.encode(sent,add_special_tokens=True)
    tokens_t.append(input_ids)
    length_of_tokens_t.append(len(input_ids))
  dataset['tokens_t']= tokens_t   
  dataset['length_of_tokens_t']= length_of_tokens_t 
  print("")
  print("token length of the target sentences")
  display(dataset.length_of_tokens_t.describe())



  tokens_ct=[]
  length_of_tokens_ct=[]
  for sent1,sent2 in zip(dataset.context,dataset.target):
    input_ids = tokenizer.encode(sent1,sent2,add_special_tokens=True)
    tokens_ct.append(input_ids)
    length_of_tokens_ct.append(len(input_ids))
  dataset['tokens_ct']= tokens_ct   
  dataset['length_of_tokens_ct']= length_of_tokens_ct 
  print("")
  print("token length of the context and target sentences")
  display(dataset.length_of_tokens_ct.describe())

  display(dataset.head(4))


Loading BERT tokenizer...


In [4]:
display_dataset(train_data)

Look at example representative entries of the dataset


,idx,label,context,target
0,0,2,The UK is fucked.,>The ~~UK~~ world is fucked FTFY
1,1,0,Listen to this wisdom.,Where the Fuck did you get that up arrow?



Is the dataset complete or are some values missing?


idx        3325
label      3325
context    3325
target     3325
dtype: int64


How balanced is the dataset?


...


token length of the target sentences


count    3325.000000
mean       16.517594
std         7.609138
min         7.000000
25%        12.000000
50%        16.000000
75%        20.000000
max       191.000000
Name: length_of_tokens_t, dtype: float64


token length of the context and target sentences


count    3325.000000
mean       73.890827
std        49.148959
min        15.000000
25%        39.000000
50%        59.000000
75%        93.000000
max       656.000000
Name: length_of_tokens_ct, dtype: float64

,idx,label,context,target,tokens_c,length_of_tokens_c,tokens_t,length_of_tokens_t,tokens_ct,length_of_tokens_ct
0,0,2,The UK is fucked.,>The ~~UK~~ world is fucked FTFY,"[101, 1996, 2866, 2003, 21746, 1012, 102]",7,"[101, 1028, 1996, 1066, 1066, 2866, 1066, 1066...",14,"[101, 1996, 2866, 2003, 21746, 1012, 102, 1028...",20
1,1,0,Listen to this wisdom.,Where the Fuck did you get that up arrow?,"[101, 4952, 2000, 2023, 9866, 1012, 102]",7,"[101, 2073, 1996, 6616, 2106, 2017, 2131, 2008...",12,"[101, 4952, 2000, 2023, 9866, 1012, 102, 2073,...",18
2,2,1,"""That's different.""",You're right. The ultrasound says it's a girl....,"[101, 1000, 2008, 1005, 1055, 2367, 1012, 1000...",9,"[101, 2017, 1005, 2128, 2157, 1012, 1996, 2731...",23,"[101, 1000, 2008, 1005, 1055, 2367, 1012, 1000...",31
3,3,2,"Oh fuck offffff, this is just patently untrue ...",Lol you can always spot a valid arguement when...,"[101, 2821, 6616, 2125, 4246, 4246, 1010, 2023...",40,"[101, 8840, 2140, 2017, 2064, 2467, 3962, 1037...",21,"[101, 2821, 6616, 2125, 4246, 4246, 1010, 2023...",60


In [5]:
display_dataset(val_data)

Look at example representative entries of the dataset


,idx,label,context,target
0,0,2,The fact that you think that is sufficient for...,Not being able to find a job for 20 years soun...
1,1,2,Because it's not true you fucking liar. Not ev...,Can't handle the truth hmmmm?



Is the dataset complete or are some values missing?


idx        713
label      713
context    713
target     713
dtype: int64


How balanced is the dataset?


...


token length of the target sentences


count    713.000000
mean      16.468443
std        6.619424
min        7.000000
25%       12.000000
50%       16.000000
75%       20.000000
max       76.000000
Name: length_of_tokens_t, dtype: float64


token length of the context and target sentences


count    713.000000
mean      73.890603
std       45.338234
min       15.000000
25%       41.000000
50%       60.000000
75%       96.000000
max      268.000000
Name: length_of_tokens_ct, dtype: float64

,idx,label,context,target,tokens_c,length_of_tokens_c,tokens_t,length_of_tokens_t,tokens_ct,length_of_tokens_ct
0,0,2,The fact that you think that is sufficient for...,Not being able to find a job for 20 years soun...,"[101, 1996, 2755, 2008, 2017, 2228, 2008, 2003...",106,"[101, 2025, 2108, 2583, 2000, 2424, 1037, 3105...",18,"[101, 1996, 2755, 2008, 2017, 2228, 2008, 2003...",123
1,1,2,Because it's not true you fucking liar. Not ev...,Can't handle the truth hmmmm?,"[101, 2138, 2009, 1005, 1055, 2025, 2995, 2017...",17,"[101, 2064, 1005, 1056, 5047, 1996, 3606, 1701...",11,"[101, 2138, 2009, 1005, 1055, 2025, 2995, 2017...",27
2,2,1,"What about the meme ""lol male tears""?",Male tears is spunk lmfao,"[101, 2054, 2055, 1996, 2033, 4168, 1000, 8840...",14,"[101, 3287, 4000, 2003, 7455, 2243, 1048, 2213...",11,"[101, 2054, 2055, 1996, 2033, 4168, 1000, 8840...",24
3,3,1,This is not a feminist issue but a capitalist ...,Exactly what part of this is a capitalist issue?,"[101, 2023, 2003, 2025, 1037, 10469, 3277, 202...",120,"[101, 3599, 2054, 2112, 1997, 2023, 2003, 1037...",12,"[101, 2023, 2003, 2025, 1037, 10469, 3277, 202...",131


In [6]:
display_dataset(test_data)

Look at example representative entries of the dataset


,idx,label,context,target
0,0,2,Someone on Tumblr actually complied a list and...,Can I get a link to that?
1,1,1,"She has no trouble using Feminism to help her,...",Rand Paul thinks so too. Maybe you should vote...



Is the dataset complete or are some values missing?


idx        713
label      713
context    713
target     713
dtype: int64


How balanced is the dataset?


...


token length of the target sentences


count    713.000000
mean      15.725105
std        5.498228
min        7.000000
25%       12.000000
50%       15.000000
75%       19.000000
max       55.000000
Name: length_of_tokens_t, dtype: float64


token length of the context and target sentences


count    713.000000
mean      74.071529
std       49.660015
min       14.000000
25%       39.000000
50%       57.000000
75%       93.000000
max      271.000000
Name: length_of_tokens_ct, dtype: float64

,idx,label,context,target,tokens_c,length_of_tokens_c,tokens_t,length_of_tokens_t,tokens_ct,length_of_tokens_ct
0,0,2,Someone on Tumblr actually complied a list and...,Can I get a link to that?,"[101, 2619, 2006, 10722, 14905, 20974, 2941, 2...",42,"[101, 2064, 1045, 2131, 1037, 4957, 2000, 2008...",10,"[101, 2619, 2006, 10722, 14905, 20974, 2941, 2...",51
1,1,1,"She has no trouble using Feminism to help her,...",Rand Paul thinks so too. Maybe you should vote...,"[101, 2016, 2038, 2053, 4390, 2478, 20050, 200...",57,"[101, 14566, 2703, 6732, 2061, 2205, 1012, 267...",15,"[101, 2016, 2038, 2053, 4390, 2478, 20050, 200...",71
2,2,1,It varies from state to state but generally sp...,Here in Florida we have the Romeo and Juliet ...,"[101, 2009, 9783, 2013, 2110, 2000, 2110, 2021...",136,"[101, 2182, 1999, 3516, 2057, 2031, 1996, 1239...",19,"[101, 2009, 9783, 2013, 2110, 2000, 2110, 2021...",154
3,3,1,You are literally using it as meme. How can yo...,Seriously OP is a moron for multiple reasons.,"[101, 2017, 2024, 6719, 2478, 2009, 2004, 2033...",28,"[101, 5667, 6728, 2003, 1037, 22822, 2239, 200...",12,"[101, 2017, 2024, 6719, 2478, 2009, 2004, 2033...",39


In [7]:
from torch.utils.data import TensorDataset
def tokenize(train_data):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  X = []
  for context, target in zip(train_data.context, train_data.target):
    encoded_dict = tokenizer.encode_plus(
                          context, target,                      # Sentence to encode.
                          add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                          max_length = 512,           # Pad & truncate all sentences.
                          padding='max_length',
                          truncation=True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

  
  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  print(input_ids)
  print(len(input_ids))
  attention_masks = torch.cat(attention_masks, dim=0)
  print(attention_masks)
  print(len(attention_masks))
  labels = torch.tensor(train_data.label)

  # Print sentence 0, now as a list of IDs.
  #print('Original: ', train_data.context[0]+train_data.target[0])
  #print('Token IDs:', input_ids[0])  
  #print('Attention_mask:', attention_masks[0])  
  #print('Label:', labels[0])
  dataset = TensorDataset(input_ids, attention_masks, labels)

  return dataset

In [8]:
# Create torch dataset
#need a dictionary
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    
    def __getitem__(self, idx):
      item = self.encodings[idx]
      return item

    def __len__(self):
        return len(self.encodings)



In [9]:
train_dataset = tokenize(train_data)
val_dataset = tokenize(val_data)
test_dataset = tokenize(test_data)
len(train_dataset)

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


tensor([[  101,  1996,  2866,  ...,     0,     0,     0],
        [  101,  4952,  2000,  ...,     0,     0,     0],
        [  101,  1000,  2008,  ...,     0,     0,     0],
        ...,
        [  101,  2009,  1005,  ...,     0,     0,     0],
        [  101,  1045,  1005,  ...,     0,     0,     0],
        [  101,  2061, 10468,  ...,     0,     0,     0]])
3325
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
3325
tensor([[ 101, 1996, 2755,  ...,    0,    0,    0],
        [ 101, 2138, 2009,  ...,    0,    0,    0],
        [ 101, 2054, 2055,  ...,    0,    0,    0],
        ...,
        [ 101, 2017, 1005,  ...,    0,    0,    0],
        [ 101, 1045, 2514,  ...,    0,    0,    0],
        [ 101, 1028, 1045,  ...,    0,    0,    0]])
713
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 

3325

In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )
print(len(train_dataloader))
# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

208


In [11]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    current_model, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 3, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [12]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [13]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8  # args.adam_epsilon  - default is 1e-8.
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 10

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [15]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [16]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [17]:
import random
import numpy as np
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4

======== Epoch 1 / 10 ========
Training...
  Batch    40  of    208.    Elapsed: 0:01:00.
  Batch    80  of    208.    Elapsed: 0:01:58.
  Batch   120  of    208.    Elapsed: 0:02:57.
  Batch   160  of    208.    Elapsed: 0:03:56.
  Batch   200  of    208.    Elapsed: 0:04:55.

  Average training loss: 1.02
  Training epcoh took: 0:05:06

Running Validation...
  Accuracy: 0.57
  Validation Loss: 0.96
  Validation took: 0:00:25

======== Epoch 2 / 10 ========
Training...
  Batch    40  of    208.    Elapsed: 0:00:58.
  Batch    80  of    208.    Elapsed: 0:01:57.
  Batch   120  of    208.    Elapsed: 0:02:56.
  Batch   160  of    208.    Elapsed: 0:03:54.
  Batch   200  of    208.    Elapsed: 0:04:53.

  Average training loss: 0.80
  Training epcoh took: 0:05:04

Running Validation...
  Accuracy: 0.59
  Validation Loss: 0.95
  Validation took: 0:00:25

======== Epoch 3 / 10 ========
Training...
  Ba

In [19]:


# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,1.018581,0.955789,0.566204,0:05:06,0:00:25
2,0.800851,0.951006,0.585648,0:05:04,0:00:25
3,0.438111,1.211186,0.578704,0:05:04,0:00:25
4,0.227260,1.661598,0.569599,0:05:05,0:00:25
5,0.107943,2.598197,0.553704,0:05:05,0:00:25
6,0.056046,3.099112,0.539043,0:05:04,0:00:25
7,0.019980,3.473183,0.525463,0:05:04,0:00:25
8,0.007111,3.457254,0.552932,0:05:05,0:00:25
9,0.000697,3.476369,0.558488,0:05:05,0:00:25


In [ ]:
import matplotlib.pyplot as plt


import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-5c45ad6da4e9>", line 4, in <cell line: 4>
    import seaborn as sns
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1002, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 945, in _find_spec
  File "<frozen importlib._bootstrap_external>", line 1439, in find_spec
  File "<frozen importlib._bootstrap_external>", line 1408, in _get_spec
  File "<frozen importlib._bootstrap_external>", line 1366, in _path_importer_cache
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceba